In [ ]:
 import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/AD Identification using SATD'

Mounted at /content/drive
/content/drive/My Drive/AD Identification using SATD


In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder, StandardScaler, MultiLabelBinarizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline



#Read the dataset


In [ ]:
import pandas as pd
liu_ = '/content/drive/My Drive/AD Identification using SATD/liu_datset_processed.csv'
liu_ = pd.read_csv(liu_, low_memory=False)

In [ ]:
liu_['Comments'].fillna('', inplace=True)
liu_['TDType'] = liu_['TDType'].astype(str)

values_to_remove = ['MULTITHREAD', 'nan', 'removeType']
replacement_value = 'WITHOUT_CLASSIFICATION'
liu_['TDType'].replace(values_to_remove, replacement_value, inplace=True)

liu_['Comments'] = liu_['Comments'].str.replace('content=', '', regex=False)
liu_['Comments'] = liu_['Comments'].str.replace('"', '', regex=False)

In [ ]:
liu_ = liu_.drop_duplicates(subset=['Comments', 'TDType'])
liu_['TDType'] = liu_['TDType'].replace('removeType', 'WITHOUT_CLASSIFICATION')

In [ ]:
# Count the number of duplicate rows in the DataFrame
num_duplicates = liu_.duplicated().sum()

print(f"Number of duplicate rows: {num_duplicates}")


Number of duplicate rows: 0


##Hash

In [ ]:
data = liu_

# Split data into train and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(data['Comments'], data['TDType'], test_size=0.2, random_state=42)
# Further split the training set into training and validation sets
X_train_final, X_val, y_train_final, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Define the pipeline
pipeline = Pipeline([
    ('HASH', HashingVectorizer()),
    ('scaler', StandardScaler(with_mean=False)),  # Scale feature vectors
    ('clf', SVC(random_state=42, kernel='linear', class_weight='balanced', probability=True))  # SVM
])


param_grid = {
    #'clf__C': [0.01, 0.1, 1, 10],  # Regularization strength
    'clf__gamma': ['scale', 'auto']  # Kernel coefficient for 'rbf'
}

# Set up GridSearchCV with StratifiedKFold and the parameter grid
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
                           scoring='accuracy',
                           n_jobs=-1,
                           verbose=2)

# Fit the grid search to the training data
grid_search.fit(X_train_final, y_train_final)

# Print the best parameters found by GridSearchCV
print("Best parameters found:", grid_search.best_params_)

# Evaluate the best model on the validation set
best_model = grid_search.best_estimator_
y_val_pred = best_model.predict(X_val)
conf_matrix_val = confusion_matrix(y_val, y_val_pred)
classification_rep_val = classification_report(y_val, y_val_pred)

print("\nValidation Confusion Matrix:")
print(conf_matrix_val)
print("\nValidation Classification Report:")
print(classification_rep_val)

# Evaluate the best model on the test set
y_test_pred = best_model.predict(X_test)
conf_matrix_test = confusion_matrix(y_test, y_test_pred)
classification_rep_test = classification_report(y_test, y_test_pred)

print("\nTest Confusion Matrix:")
print(conf_matrix_test)
print("\nTest Classification Report:")
print(classification_rep_test)


Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best parameters found: {'clf__gamma': 'scale'}

Validation Confusion Matrix:
[[  49    0    2   56    0    3    0   33]
 [   1   26    2   23    0    3    0   12]
 [   1    4   43   40    1    1    3    6]
 [  46   12   44 1313    3   91    6  201]
 [   0    0    0    8    9    1    0    2]
 [   2    4    7  109    3  194    1   28]
 [   3    0    4   29    0    4   55   10]
 [  36    6    8  182    0    9    1 3480]]

Validation Classification Report:
                        precision    recall  f1-score   support

             ALGORITHM       0.36      0.34      0.35       143
         COMPATIBILITY       0.50      0.39      0.44        67
                DEFECT       0.39      0.43      0.41        99
                DESIGN       0.75      0.77      0.76      1716
         DOCUMENTATION       0.56      0.45      0.50        20
        IMPLEMENTATION       0.63      0.56      0.59       348
                  TEST       0.83 